# Advent of Code 2022

## [Day 10](https://adventofcode.com/2022/day/10)

Task:

* Execute a series of instructions and track changes in a value according to those instructions
* Then compare the value at particular points in time to another value

Reflection:

The biggest thing that made this tricky were a bunch of off-by-one errors. The timing cycles start at 1, the pixel values start at 0, and any list of values will be 0-indexed. It ended up being confusing trying to figure out where ranges started, and at what values the pixel value should reset. I initially tried setting the pixel value with modulo division, but could not get it to work correctly.

In [489]:
import aoc
aoc.read_input("cookie.txt", "input_2022", 2022, 10)

with open("input_2022/10.txt") as f:
    instructions = [line.split() for line in f.read().strip().split("\n")]


### puzzle 1

In [491]:
x = 1
cycle = 0
cycleVals = [1]
for instruction in instructions:
    if instruction[0] == 'noop':
        cycle += 1
        cycleVals.append(x)
    elif instruction[0] == 'addx':
        cycle += 1
        cycleVals.append(x)
        cycle += 1
        cycleVals.append(x)
        x += int(instruction[1])
        
interestingCycles = range(20,221,40)
interestingCycleVals = cycleVals[20::40]
sum([c * cv for c, cv in zip(interestingCycles, interestingCycleVals)])


13860

## puzzle 2

In [490]:
x = 1
cycle = 0
cycleVals = [1]

for instruction in instructions:
    if instruction[0] == 'noop':
        cycle += 1
        cycleVals.append(x)
    elif instruction[0] == 'addx':
        cycle += 1
        cycleVals.append(x)
        cycle += 1
        cycleVals.append(x)
        x += int(instruction[1])


sprite_pos = [[pos-1, pos, pos+1] for pos in cycleVals]
lines = []
line = ""
pixel = 0
for cycle in range(1,241):
    if pixel in sprite_pos[cycle]:
        line += "#"
    else:
        line += "."
    pixel += 1
    if pixel == 40:
        linecopy = line
        lines.append(linecopy)
        line = ""
        pixel = 0
    
for line in lines:
    print(line)

###..####.#..#.####..##....##..##..###..
#..#....#.#..#.#....#..#....#.#..#.#..#.
#..#...#..####.###..#.......#.#....###..
###...#...#..#.#....#.##....#.#....#..#.
#.#..#....#..#.#....#..#.#..#.#..#.#..#.
#..#.####.#..#.#.....###..##...##..###..


## [Day 9]( https://adventofcode.com/2022/day/9)

Task:

* From a set of instructions, track the movement of an object in 2D space
* Then track an object following the first
* Then track a series of objects, each following the one in front of it

Reflection

This one was hard to code gracefully. There is a lot of repeat copy/paste code that could undoubtedly be refactored into something more elegant. The code for puzzle 2 should have worked recursively, but for whatever reason I couldn't get it to work that way. Manually repeating the function with the previous tails input did get the right answer. Very ugly stuff.

In [289]:
import aoc
aoc.read_input("cookie.txt", "input_2022", 2022, 9)

### puzzle 1

In [397]:
hCoords = [0, 0]
tCoords = [0, 0]
tpos = [tCoords]
with open("input_2022/9_test.txt") as f:
    path = [line.split() for line in f.read().split("\n")]
    
    for step in path:
        step[1] = int(step[1])
        if step[0] == 'R':
            for i in range(step[1]):
                hCoords[0] += 1
                if abs(hCoords[0] - tCoords[0]) > 1 or abs(hCoords[1] - tCoords[1]) > 1:
                    tCoords = [hCoords[0] - 1, hCoords[1]]
                    tpos.append(tCoords)

        elif step[0] == 'L':
            for i in range(step[1]):
                hCoords[0] -= 1
                if abs(hCoords[0] - tCoords[0]) > 1 or abs(hCoords[1] - tCoords[1]) > 1:
                    tCoords = [hCoords[0] + 1, hCoords[1]]
                    tpos.append(tCoords)

        elif step[0] == 'U':
            for i in range(step[1]):
                hCoords[1] += 1
                if abs(hCoords[0] - tCoords[0]) > 1 or abs(hCoords[1] - tCoords[1]) > 1:
                    tCoords = [hCoords[0], hCoords[1] - 1]
                    tpos.append(tCoords)
            
        else:
            for i in range(step[1]):
                hCoords[1] -= 1
                if abs(hCoords[0] - tCoords[0]) > 1 or abs(hCoords[1] - tCoords[1]) > 1:
                    tCoords = [hCoords[0], hCoords[1] + 1]
                    tpos.append(tCoords)

print(tpos)
len(set([tuple(coords) for coords in tpos]))

[[0, 0], [1, 0], [2, 0], [3, 0], [4, 1], [4, 2], [4, 3], [3, 4], [2, 4], [3, 3], [4, 3], [3, 2], [2, 2], [1, 2]]


13

### puzzle 2

the head can only move rook's case, so if it gets too far away from the first tail, that tail can just update to head's last position. That's worked in the first puzzle. But the first tail can move diagonally, so that trick doesn't work for second and subsequent tails.

In [434]:
hpos = [[0, 0]]
hCoords = hpos[0]


with open("input_2022/9.txt") as f:
    path = [line.split() for line in f.read().split("\n")]
    
    # get head positions
    for step in path:
        step[1] = int(step[1])
        for i in range(step[1]):
            hCoords = hCoords.copy()
            hpos.append(hCoords)
            if step[0] == 'R':
                hCoords[0] += 1
            
            elif step[0] == 'L':
                hCoords[0] -= 1

            elif step[0] == 'U':
                hCoords[1] += 1
                
            else:
                hCoords[1] -= 1

import copy
# get tail positions from head positions:
def tail_pos_tracker(hpos, knot_num):
    tpos = []
    tCoords = [0,0]
    while knot_num:
        
        knot_num -= 1
        for i, hCoords in enumerate(hpos):
            tCoords = tCoords.copy()
            tpos.append(tCoords)
            # 8 scenarios
            # h is 2 spaces right
            if hCoords[0] - tCoords[0] == 2 and hCoords[1] == tCoords[1]:
                tCoords[0] += 1
            
            # h is 2 spaces left
            if hCoords[0] - tCoords[0] == -2 and hCoords[1] == tCoords[1]:
                tCoords[0] -= 1

            # h is 2 spaces up
            if hCoords[1] - tCoords[1] == 2 and hCoords[0] == tCoords[0]:
                tCoords[1] += 1

            # h is 2 spaces down
            if hCoords[1] - tCoords[1] == -2 and hCoords[0] == tCoords[0]:
                tCoords[1] -= 1

            # h is (2 up, 1 right) or (1 up 2 right) or (2 up 2 right)
            if (hCoords[1] - tCoords[1] == 2 and hCoords[0] - tCoords[0] == 1) or (hCoords[1] - tCoords[1] == 1 and hCoords[0] - tCoords[0] == 2) or (hCoords[1] - tCoords[1] == 2 and hCoords[0] - tCoords[0] == 2):
                tCoords[0] += 1
                tCoords[1] += 1

            # h is (2 up, 1 left) or (1 up 2 left) or (2 up 2 left)
            if (hCoords[1] - tCoords[1] == 2 and hCoords[0] - tCoords[0] == -1) or (hCoords[1] - tCoords[1] == 1 and hCoords[0] - tCoords[0] == -2) or (hCoords[1] - tCoords[1] == 2 and hCoords[0] - tCoords[0] == -2):
                tCoords[0] -= 1
                tCoords[1] += 1

            # h is (2 down, 1 left) or (1 down, 2 left) or (2 down, 2 left)
            if (hCoords[1] - tCoords[1] == -2 and hCoords[0] - tCoords[0] == -1) or (hCoords[1] - tCoords[1] == -1 and hCoords[0] - tCoords[0] == -2) or (hCoords[1] - tCoords[1] == -2 and hCoords[0] - tCoords[0] == -2):
                tCoords[0] -= 1
                tCoords[1] -= 1

            # h is (2 down, 1 right) or (1 down, 2 right) or (2 down, 2 right)
            if (hCoords[1] - tCoords[1] == -2 and hCoords[0] - tCoords[0] == 1) or (hCoords[1] - tCoords[1] == -1 and hCoords[0] - tCoords[0] == 2) or (hCoords[1] - tCoords[1] == -2 and hCoords[0] - tCoords[0] == 2):
                tCoords[0] += 1
                tCoords[1] -= 1
        tail_pos_tracker(copy.deepcopy(tpos), knot_num)
        
    return tpos

tpos = tail_pos_tracker(hpos, 1)
t2pos = tail_pos_tracker(tpos, 1)
t3pos = tail_pos_tracker(t2pos, 1)
t4pos = tail_pos_tracker(t3pos, 1)
t5pos = tail_pos_tracker(t4pos, 1)
t6pos = tail_pos_tracker(t5pos, 1)
t7pos = tail_pos_tracker(t6pos, 1)
t8pos = tail_pos_tracker(t7pos, 1)
t9pos = tail_pos_tracker(t8pos, 1)

len(set([tuple(coords) for coords in t9pos]))

2427

## [Day 8](https://adventofcode.com/2022/day/8)

Task: 

* For each element in a 2D array, compare it to the other values in its row and column

Reflection

The solution to the first puzzle is shamefully bad. A bunch of copy/paste code that could have been a single function. The solution to the second puzzle is cleaner, mostly becuase I figured out out to get both the row and column while iterating through every cell value. Instead of iterating through the individual scalar values in the array, though, it might be interesting to apply a single function vector-wise.

In [144]:
import aoc
aoc.read_input("cookie.txt", "input_2022", 2022, 8)

### puzzle 1

In [281]:
import numpy as np
with open("input_2022/8.txt") as f:
    data = [[float(char) for char in seq] for seq in f.read().strip().split("\n")]
    data = np.pad(data, pad_width=1, mode='constant', constant_values=-1)

vis_from_left = data.copy()
for i, row in enumerate(data[1:-1]):
    for j, cell in enumerate(row[1:-1]):
        left_vals = row[0:j+1]
        vis_from_left[i+1, j+1] =  cell > max(left_vals)
vis_from_right = data.copy()
for i, row in enumerate(data[1:-1]):
    for j, cell in enumerate(row[1:-1]):
        right_vals = row[j+2:]
        vis_from_right[i+1, j+1] =  cell > max(right_vals)
vis_from_top = data.copy()
for j, column in enumerate(data.T[1:-1]):
    for i, cell in enumerate(column[1:-1]):
        top_vals = column[0:i+1]
        vis_from_top[i+1, j+1] =  cell > max(top_vals)
vis_from_bottom = data.copy()
for j, column in enumerate(data.T[1:-1]):
    for i, cell in enumerate(column[1:-1]):
        bottom_vals = column[i+2:]
        vis_from_bottom[i+1, j+1] =  cell > max(bottom_vals)


((vis_from_left + vis_from_top + vis_from_bottom + vis_from_right) > 0).sum()

1733

### puzzle 2

In [288]:
with open("input_2022/8.txt") as f:
    data = [[int(char) for char in seq] for seq in f.read().strip().split("\n")]
    data = np.array(data)

def get_view_count(view, height):
    count = 0
    for val in view:
        count += 1
        if height <= val:
            break
    return count

scoreArray = data.copy()
for i, row in enumerate(data):
    for j, cell in enumerate(row):
        column = data[:,j]
        right = row[j+1:]
        left = row[:j][::-1]
        bottom = column[i+1:]
        top = column[:i][::-1]
        score = 1
        for view in (right, left, bottom, top):
            view_count = get_view_count(view, cell)
            score *= view_count
        scoreArray[i, j] = score
scoreArray.max()

284648

## [Day 7](https://adventofcode.com/2022/day/7)

Task: 

* Parse a set of terminal commands and their output into a directory structure
* Calculate the size of each directory

Reflection

I went back and forth between using a networkx graph and using a nested dictionary to model the directory structure. I initially ran into trouble with the graph because I was only tracking the directory name, and there are different directories with the same name (i.e. they have different parents). Tracking only the name of the directory meant that those multiple directories were treated a single node. The key was to use a directed graph to understand the parent/child relationship, and create unique node names that tracked the entire directory path.

In [17]:
import aoc
aoc.read_input("cookie.txt", "input_2022", 2022, 7)

### puzzle 1

In [142]:
import networkx as nx
G = nx.DiGraph()
dir_list = [("/",)]
dir_path = ()
with open("input_2022/7.txt") as f:
    for line in f:
        line = line.strip()
        if line.startswith("$ cd"):
            if not line.endswith(".."):
                current_dir = line.split()[-1]
                dir_path += (current_dir,)
            else:
                dir_path = dir_path[:-1]
        elif line == "$ ls":
            continue
        elif line.startswith("dir"):
            G.add_edge(dir_path, dir_path + (line.split()[-1], ))
            dir_list.append(dir_path + (line.split()[-1], ))
        else:
            G.add_edge(dir_path, line.split()[0])

# if G is not a tree, something went wrong
nx.is_tree(G)

dir_sizes = []
for dir in dir_list:
    size = sum([int(d) for d in nx.descendants(G, dir) if d[-1].isnumeric()])
    if size <= 100000:
        dir_sizes.append(size)
sum(dir_sizes)


919137

### puzzle 2

In [143]:
dir_sizes = []
for dir in dir_list:
    size = sum([int(d) for d in nx.descendants(G, dir) if d[-1].isnumeric()])
    dir_sizes.append(size)
total = max(dir_sizes)
available = 70000000 - total
needed = 30000000 - available
possible_dirs = []
for dir_size in dir_sizes:
    if dir_size >= needed:
        possible_dirs.append(dir_size)
min(possible_dirs)

2877389

## [Day 6](https://adventofcode.com/2022/day/6)

Task:

* Evaluate a substring created by a moving window through a string.

Reflection

At first I thought that it would be best to utilize some type of `window` function in a library. But then realized that creating my own window function was fairly easy, and faster than searching for a library with a built-in one.



In [2]:
import aoc
aoc.read_input("cookie.txt", "input_2022", 2022, 6)

### puzzle 1

In [12]:
with open("input_2022/6.txt") as f:
    txt = f.read()
    for i, char in enumerate(txt):
        if len(set(txt[i:i+4])) == 4:
            print(i +4)
            break

1707


### puzzle 2

In [16]:
with open("input_2022/6.txt") as f:
    txt = f.read()
    for i, char in enumerate(txt):
        if len(set(txt[i:i+14])) == 14:
            print(i + 14)
            break

3697


## [Day 5](https://adventofcode.com/2022/day/5)

Task:

* Move elements from one column to another. First one at a time, then in a group

Reflection:

As with other puzzles so far, the hard part has been getting the text input into a form that can be operated on. In this case, I transformed the columns into seperate lists, and the instructions into a dictionary. Once that was done, I could pop (or slice) the elements off the end of one list and append (or extend) some other list.

In [15]:
import aoc
aoc.read_input("cookie.txt", "input_2022", 2022, 5)

### puzzle 1

In [113]:
import numpy as np
instructions = []
with open("input_2022/5.txt") as f:
    rowLists = []
    for line in f:
        if line.startswith(" 1") or line.startswith("\n"):
            continue
        elif line.startswith("move"):
            instructionText = line.strip().split()
            instructionDict = {
                "num":int(instructionText[1]),
                "from_column": int(instructionText[3]) - 1,
                "to_column": int(instructionText[5]) - 1
            }
            instructions.append(instructionDict)
        else:
            row = line.rstrip("\n")
            rowSplits = [i for i in range(0,len(row), 4)]
            rowList = [row[i:i+3] for i in rowSplits]
            rowLists.append(rowList)
rowArray  = np.array(rowLists)
columns = []
num_columns = rowArray.shape[1]
for column in range(num_columns):
    column_raw = rowArray[:,column]
    column_trimmed = column_raw[column_raw != "   "].tolist()
    column_trimmed.reverse()
    columns.append(column_trimmed)

for instruction in instructions:
    for i in range(instruction['num']):
        crate = columns[instruction['from_column']].pop()
        columns[instruction['to_column']].append(crate)
"".join([column[-1][1] for column in columns])

'RLFNRTNFB'

### puzzle 2

In [119]:
import numpy as np
instructions = []
with open("input_2022/5.txt") as f:
    rowLists = []
    for line in f:
        if line.startswith(" 1") or line.startswith("\n"):
            continue
        elif line.startswith("move"):
            instructionText = line.strip().split()
            instructionDict = {
                "num":int(instructionText[1]),
                "from_column": int(instructionText[3]) - 1,
                "to_column": int(instructionText[5]) - 1
            }
            instructions.append(instructionDict)
        else:
            row = line.rstrip("\n")
            rowSplits = [i for i in range(0,len(row), 4)]
            rowList = [row[i:i+3] for i in rowSplits]
            rowLists.append(rowList)
rowArray  = np.array(rowLists)
columns = []
num_columns = rowArray.shape[1]
for column in range(num_columns):
    column_raw = rowArray[:,column]
    column_trimmed = column_raw[column_raw != "   "].tolist()
    column_trimmed.reverse()
    columns.append(column_trimmed)

for instruction in instructions:
    crates = columns[instruction['from_column']][-1*instruction['num']:]
    columns[instruction['from_column']] = columns[instruction['from_column']][:-1*instruction['num']]
    columns[instruction['to_column']].extend(crates)
"".join([column[-1][1] for column in columns])

'MHQTLJRLB'

# [Day 4](https://adventofcode.com/2022/day/3)

Task:

* Find overlaps between two ranges of numbers

Reflection:

I basically never use regex. This is a good use case of a simple pattern that allowed the input to be split on a `,` or a `-`. Otherwise I would have had to split the input twice, making sure to avoid nested lists. Once again, set intersections proved very useful.

In [1]:
import aoc
aoc.read_input("cookie.txt", "input_2022", 2022, 4)

### puzzle 1

In [13]:
import re
total_overlapping = 0
with open("input_2022/4.txt") as f:
    for line in f:
        start1, stop1, start2, stop2 = [int(val) for val in re.split(',|-', line.strip())]
        range1 = set(range(start1, stop1+1))
        range2 = set(range(start2, stop2+1))
        intersect = set.intersection(range1, range2)
        if len(intersect) == len(range1) or len(intersect) == len(range2):
            total_overlapping += 1
total_overlapping


477

### puzzle 2

In [14]:
import re
total_overlapping = 0
with open("input_2022/4.txt") as f:
    for line in f:
        start1, stop1, start2, stop2 = [int(val) for val in re.split(',|-', line.strip())]
        range1 = set(range(start1, stop1+1))
        range2 = set(range(start2, stop2+1))
        intersect = set.intersection(range1, range2)
        if len(intersect) > 0:
            total_overlapping += 1
total_overlapping

830

## [Day 3](https://adventofcode.com/2022/day/3)

Task: 

* Find the single character that is shared between a collection of strings
* Get a numeric value for the character according (index in a priority order)
* Sum value across a set of string collections

Reflection: 

Somthing new I learned is that the easiest way to find which elements are shared between two collections (including a string)  is to find the intersection of their sets. This won't help if you need to only find matches in the same position, or if you need to count the total number of shared elements (regardless of repeating shared elements).

### puzzle 1

In [112]:
import string
priority_order = "0" + string.ascii_lowercase + string.ascii_uppercase
priority_sum = 0
with open("input/3.txt") as f:
    for line in f:
        c1 = line[:len(line)//2]
        c2 = line[len(line)//2:]
        common = set(c1).intersection(c2).pop()
        priority_sum += priority_order.index(common)
priority_sum

7821

### puzzle 2

In [113]:
import string
priority_order = "0" + string.ascii_lowercase + string.ascii_uppercase
priority_sum = 0
with open("input/3.txt") as f:
    group = []
    for i, line in enumerate(f):
        group.append(line.rstrip())
        if i % 3 == 2:
            badge = set(group[0]).intersection(group[1]).intersection(group[2]).pop()
            priority_sum += priority_order.index(badge)
            group = []        
priority_sum        

2752

### puzzle 1 refactor

In [114]:
import string
priority_order = "0" + string.ascii_lowercase + string.ascii_uppercase
priority_sum = 0
with open("input/3.txt") as f:
    for line in f:
        c1 = set(line[:len(line)//2])
        c2 = set(line[len(line)//2:])
        common = set.intersection(c1, c2).pop()
        priority_sum += priority_order.index(common)
priority_sum

7821

### puzzle 2 refactor

In [115]:
import string
priority_order = "0" + string.ascii_lowercase + string.ascii_uppercase
priority_sum = 0
with open("input/3.txt") as f:
    group = []
    for i, line in enumerate(f):
        unique_items = set(line.rstrip())
        group.append(unique_items)
        if i % 3 == 2:
            badge = set.intersection(*group).pop()
            priority_sum += priority_order.index(badge)
            group = []        
priority_sum  

2752

## [Day 2](https://adventofcode.com/2022/day/2)

Task:

* Determine the winner of a series of Rock Paper Scissors matches. 

Reflection:

Lots of people tried to solve this with modulo division, to capture the non-transitive nature of RPS. That's a very clever solution, that I tried to implement after I had a working solution of my own. But the more I looked at their solutions, the less I liked them compared to the simplicity of my original.

### puzzle 1

In [39]:
scores = {
    "A X": 4,
    "A Y": 8,
    "A Z": 3,
    "B X": 1,
    "B Y": 5,
    "B Z": 9,
    "C X": 7,
    "C Y": 2,
    "C Z": 6,
}
score  = 0
with open("AOC_input_2.txt") as f:
    for line in f:
        score += scores[line.rstrip()]
score

11386

### puzzle 2

In [40]:
scores = {
    "A X": 3,
    "A Y": 4,
    "A Z": 8,
    "B X": 1,
    "B Y": 5,
    "B Z": 9,
    "C X": 2,
    "C Y": 6,
    "C Z": 7,
}
score  = 0
with open("AOC_input_2.txt") as f:
    for line in f:
        score += scores[line.rstrip()]
score

13600

## [Day 1](https://adventofcode.com/2022/day/1)

Task:

* Group data values from a text file and sum them
* Compare summed values to find the largest

Reflection:

* The unreadable (but working) one-liner at the end is a good illustration of why flat is better than nested. 
* The running total strategy I used for the first puzzle is more memory-efficient and has better time complexity because it doesn't have to store the entire set of values, and it doesn't have to sort them. But it is much harder to scale to the situation in puzzle 2 that asks not for the largest, but for the top 3 largest.


### puzzle 1

In [2]:
with open("AOC_input_1.txt") as f:
    most_calories = 0
    calories = 0
    for line in f:
        if not line.startswith("\n"):
            calories += int(line.rstrip())
        else:
            if calories > most_calories:
                most_calories = calories
            calories = 0
most_calories

67027

### puzzle 2

In [3]:
with open("AOC_input_1.txt") as f:
    calories_list = []
    calories = 0
    for line in f:
        if not line.startswith("\n"):
            calories += int(line.rstrip())
        else:
            calories_list.append(calories)
            calories = 0
sum(sorted(calories_list, reverse=True)[:3])

197291

### refactor puzzles

In [20]:
def calorie_counter(input_file, num_elves):
    with open(input_file) as f:
        elves = f.read().split("\n\n")
        calories_list = []
        for elf in elves:
            calories = sum([int(calories) for calories in elf.split("\n") if calories])
            calories_list.append(calories)
    return sum(sorted(calories_list, reverse=True)[:num_elves])
    
calorie_counter("AOC_input_1.txt", 3)

197291

In [33]:
with open("AOC_input_1.txt") as f:
    elves = f.read().split("\n\n")
    calories_list = []
    for elf in elves:
        food_item_calories = [int(calories) for calories in elf.split("\n") if calories]
        elf_total_calories = sum(food_item_calories)
        calories_list.append(elf_total_calories)
sorted_elves = sorted(calories_list, reverse=True)
sum(sorted_elves[:3])

197291

In [35]:
sum(sorted([sum([int(c) for c in e.split("\n") if c]) for e in open("f.txt").read().split("\n\n")], reverse=True)[:1])

67027